<a href="https://colab.research.google.com/github/phpaivamotta/fruit-recognition-deep-learning/blob/main/Copy_of_Bootcamp_Draft_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Entrar no diretório

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Deep Learning em Python/Material/')

## Leitura dos arquivos

In [ ]:
import glob
import cv2
import random
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
fruit_types = {'Apple':0, 'Banana':1, 'Pear':2, 'Pineapple':3, 'Grape':4, 'Cherry':5, 'Peach':6, 'Lemon':7, 'Cocos':8, 'Mango':9, 'Watermelon':10, 'Orange':11, 'Strawberry':12}

In [ ]:
all_train_files = [img_path for img_path in glob.glob('./Train/*/*.jpg') for fruit in fruit_types if fruit in img_path]
train_files, val_files = train_test_split(all_train_files, test_size=0.2, random_state=132)

In [ ]:
len(all_train_files)

In [ ]:
len(train_files)

In [ ]:
len(val_files)

## Gerador

In [ ]:
def fruit_imgs_generator(files, batch_size=32):
  while True:
    batch_files = np.random.choice(files, size=batch_size)

    fruit_imgs = []
    fruit_labels = []
    for single_file in batch_files:
      fruit_infos = [(single_file,single_fruit_value) for single_fruit_type, single_fruit_value in fruit_types.items() if single_fruit_type in single_file]
      fruit_path = fruit_infos[0][0]
      fruit_type_value = fruit_infos[0][1]

      image = cv2.imread(fruit_path, cv2.IMREAD_COLOR)
      image = cv2.resize(image, (180,180))
                 
      image = cv2.normalize(image, 0, 1, norm_type=cv2.NORM_MINMAX)

      fruit_imgs.append(image)
      fruit_labels.append(fruit_type_value)
    
    batch_x = np.array(fruit_imgs)
    batch_y = np.array(fruit_labels)

    yield (batch_x, batch_y)

In [ ]:
batch_size = 250

In [ ]:
img_train_gen = fruit_imgs_generator(files=train_files, batch_size=batch_size)
img_val_gen = fruit_imgs_generator(files=val_files, batch_size=batch_size)

In [ ]:
img_gen = fruit_imgs_generator(train_files, batch_size=batch_size)

In [ ]:
(batch_x, batch_y) = img_gen.__next__()
(batch_x.shape, batch_y.shape)

In [ ]:
from google.colab.patches import cv2_imshow

cv2_imshow(batch_x[0])

In [ ]:
X = batch_x
y = batch_y

In [ ]:
# Sem normalizar ou a normalização já embutida no gerador!
X_train = X

Testar com modelo simples CNN

In [ ]:
from tensorflow.keras import Model
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization,Input
from keras.optimizers import Adam
from sklearn import metrics

In [ ]:
# original model
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding="same", input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=3, activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3, activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(13, activation='softmax'))

model.summary()

In [ ]:
steps_per_epoch = np.ceil(len(train_files) / batch_size)
validation_steps = np.ceil(len(val_files) / batch_size)
epochs = 4

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
history = model.fit_generator(img_train_gen, steps_per_epoch=steps_per_epoch, epochs=epochs, verbose=1, validation_data=img_val_gen, validation_steps=validation_steps)

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
# np.argmax(batch_y, axis=1)
batch_y

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training - Loss Function')

In [ ]:
np.argmax(y_pred, axis=1)

In [ ]:
y

In [ ]:
metrics.confusion_matrix(np.argmax(y_pred, axis=1), y)

Aplicar o modelo a base de teste

In [ ]:
all_test_files = [img_path for img_path in glob.glob('./Test/*.jpg')]

In [ ]:
len(all_test_files)

In [ ]:
from natsort import natsorted

In [ ]:
# organizar arquivos em ordem ascendente
sorted_test_files = natsorted(all_test_files)

In [ ]:
# checar ordem dos arquivos
for i, file_name in enumerate(sorted_test_files):
  if i != int(file_name.split('_')[-1].split('.')[0]):
    print(i, int(file_name.split('_')[-1].split('.')[0]))
    break
  else:
    print(i, file_name)

In [ ]:
fruit_types

In [ ]:
results = {}

for i, test_file_path in enumerate(sorted_test_files):
  print(test_file_path)
  image = cv2.imread(test_file_path, cv2.IMREAD_COLOR)
  image = cv2.resize(image, (180,180))
  image = cv2.normalize(image, 0, 1, norm_type=cv2.NORM_MINMAX)

  image = np.expand_dims(image, 0)

  output_class_value = model.predict(image)
  output_class_value = np.argmax(output_class_value, axis=1)
  

  results[i] = list(fruit_types.keys())[list(fruit_types.values()).index(output_class_value)]

In [ ]:
import csv

with open('arquivos_sorted_5_epochs.csv', 'w') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(['Id', 'Category'])
  for key, value in results.items():
    writer.writerow([key, value])